In [ ]:
!git clone https://github.com/imabeastdrew/Martydepth.git
%cd Martydepth

# Install the package in development mode
%pip install -e .


In [ ]:
# Install dependencies
%pip install torch wandb tqdm pyyaml

In [ ]:
# Import required libraries
import torch
import wandb
import tempfile
import json
import numpy as np
from pathlib import Path
from tqdm.notebook import tqdm
import os
import sys
import yaml

# Add project root to Python path
sys.path.append('.')

print(os.getcwd())

# Import project modules
from src.data.dataset import create_dataloader
from src.models.online_transformer import OnlineTransformer
from src.models.offline_teacher import OfflineTeacherModel
from src.evaluation.metrics import (
    calculate_harmony_metrics,
    calculate_emd_metrics,
)
from src.config.tokenization_config import (
    SILENCE_TOKEN,
    MELODY_ONSET_HOLD_START,
    CHORD_TOKEN_START,
)
from src.evaluation.evaluate_offline import generate_offline
from src.evaluation.evaluate import generate_online
from src.config.tokenization_config import PAD_TOKEN

def get_latest_model_artifact(run_id: str, model_type: str) -> tuple[Path, dict]:
    """
    Retrieve the latest model artifact and its config from wandb.
    
    Args:
        run_id: The wandb run ID to get artifacts from
        model_type: Type of model ('online', 'offline', 'discriminator', or 'contrastive')
    
    Returns:
        tuple[Path, dict]: Path to the checkpoint file and the model config
    """
    # Map model types to their artifact names
    artifact_prefix_map = {
        'online': 'online_transformer_model',
        'offline': 'offline_teacher_model',
        'discriminator': 'discriminator_reward_model',
        'contrastive': 'contrastive_reward_model'
    }
    
    prefix = artifact_prefix_map[model_type]
    artifact_name = f"{prefix}_{run_id}"
    
    # Get the latest version of the model artifact
    api = wandb.Api()
    artifact = api.artifact(f"{wandb.run.entity}/{wandb.run.project}/{artifact_name}:latest")
    
    # Download the artifact
    artifact_dir = Path("artifacts") / model_type
    artifact_dir.mkdir(parents=True, exist_ok=True)
    artifact.download(str(artifact_dir))
    
    # Find the checkpoint file
    checkpoint_file = next(artifact_dir.glob("*.pth"))
    
    # Load the checkpoint to get the config
    checkpoint = torch.load(checkpoint_file, map_location='cpu')
    config = checkpoint['config']
    
    return checkpoint_file, config

def get_tokenizer_artifact(run_id: str) -> dict:
    """
    Retrieve the tokenizer info artifact from wandb.
    
    Args:
        run_id: The wandb run ID to get artifacts from
    
    Returns:
        dict: The tokenizer info dictionary
    """
    # Get the latest version of the tokenizer artifact
    api = wandb.Api()
    artifact = api.artifact(f"{wandb.run.entity}/{wandb.run.project}/tokenizer_info_{run_id}:latest")
    
    # Download the artifact
    artifact_dir = Path("artifacts") / "tokenizer"
    artifact_dir.mkdir(parents=True, exist_ok=True)
    artifact.download(str(artifact_dir))
    
    # Load the tokenizer info
    tokenizer_file = next(artifact_dir.glob("*.json"))
    with open(tokenizer_file, 'r') as f:
        tokenizer_info = json.load(f)
    
    return tokenizer_info

In [ ]:
# Configuration
config = {
    'data_dir': 'data/interim',
    'split': 'test',
    'batch_size': 32,
    'num_workers': 4,
    'temperature': 1.0,
    'top_k': 50,
    # Specify the run IDs to evaluate
    'online_run_id': None,  # Set this to the wandb run ID of the online model
    'offline_run_id': None, # Set this to the wandb run ID of the offline model
}

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else
                     "mps" if torch.backends.mps.is_available() else
                     "cpu")
print(f"Using device: {device}")

# Initialize wandb
wandb.init(
    project="martydepth",
    name="model_evaluation",
    config=config,
    job_type="evaluation"
)

# Get model artifacts and configs
if config['online_run_id']:
    online_checkpoint_path, online_config = get_latest_model_artifact(config['online_run_id'], 'online')
    print(f"Retrieved online model checkpoint from wandb: {online_checkpoint_path}")
else:
    print("No online run ID provided, skipping online model evaluation")
    online_checkpoint_path, online_config = None, None

if config['offline_run_id']:
    offline_checkpoint_path, offline_config = get_latest_model_artifact(config['offline_run_id'], 'offline')
    print(f"Retrieved offline model checkpoint from wandb: {offline_checkpoint_path}")
else:
    print("No offline run ID provided, skipping offline model evaluation")
    offline_checkpoint_path, offline_config = None, None

# Get tokenizer info from either run (they should be the same)
run_id_for_tokenizer = config['online_run_id'] or config['offline_run_id']
if run_id_for_tokenizer:
    tokenizer_info = get_tokenizer_artifact(run_id_for_tokenizer)
    print("Retrieved tokenizer info from wandb")
else:
    # Fallback to local tokenizer info if no run IDs provided
    tokenizer_info_path = Path(config['data_dir']) / "train" / "tokenizer_info.json"
    with open(tokenizer_info_path, 'r') as f:
        tokenizer_info = json.load(f)
    print("Using local tokenizer info")

In [ ]:
# Evaluate Online Model
if online_checkpoint_path and online_config:
    print("\n=== Evaluating Online Model ===")
    print(f"Loading model from checkpoint: {online_checkpoint_path}")

    # Initialize model
    model = OnlineTransformer(
        vocab_size=tokenizer_info['total_vocab_size'],
        embed_dim=online_config['embed_dim'],
        num_heads=online_config['num_heads'],
        num_layers=online_config['num_layers'],
        dropout=online_config['dropout'],
        max_seq_length=online_config.get('max_seq_length', 512),
        pad_token_id=PAD_TOKEN # Use PAD_TOKEN here
    ).to(device)

    # Load checkpoint
    checkpoint = torch.load(online_checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()

    # Create dataloader
    max_seq_length = online_config.get('max_seq_length') or online_config.get('max_sequence_length') or 512
    dataloader, _ = create_dataloader(
        data_dir=Path(config['data_dir']),
        split=config['split'],
        batch_size=config['batch_size'],
        num_workers=config['num_workers'],
        sequence_length=max_seq_length,
        mode='online',
        shuffle=False
    )

    # Generate sequences
    print("\nGenerating sequences...")
    generated_sequences, ground_truth_sequences = generate_online(
        model=model,
        dataloader=dataloader,
        tokenizer_info=tokenizer_info,
        device=device,
        temperature=config['temperature'],
        top_k=config['top_k']
    )

    # Calculate metrics
    print("\nCalculating metrics...")
    harmony_metrics = calculate_harmony_metrics(generated_sequences, tokenizer_info)
    emd_metrics = calculate_emd_metrics(generated_sequences, ground_truth_sequences, tokenizer_info)

    online_metrics = {**harmony_metrics, **emd_metrics}

    print("\n=== Online Model Results ===")
    for metric, value in online_metrics.items():
        print(f"{metric}: {value:.4f}")
        wandb.log({f"online/{metric}": value})
        
    # Log the run ID we evaluated
    wandb.log({"online_model_run_id": config['online_run_id']})
else:
    print("\n=== Skipping Online Model Evaluation ===")
    print("No online model checkpoint provided")

In [ ]:
# Evaluate Offline Model
if offline_checkpoint_path and offline_config:
    print("\n=== Evaluating Offline Model ===")
    print(f"Loading model from checkpoint: {offline_checkpoint_path}")

    # Initialize model
    model = OfflineTeacherModel(
        melody_vocab_size=tokenizer_info['melody_vocab_size'],
        chord_vocab_size=tokenizer_info['chord_vocab_size'],
        embed_dim=offline_config['embed_dim'],
        num_heads=offline_config['num_heads'],
        num_layers=offline_config['num_layers'],
        max_seq_length=offline_config.get('max_seq_length', 512),
        pad_token_id=PAD_TOKEN # Use PAD_TOKEN here
    ).to(device)

    # Load checkpoint
    checkpoint = torch.load(offline_checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()

    # Create dataloader
    max_seq_length = offline_config.get('max_seq_length') or offline_config.get('max_sequence_length') or 512
    dataloader, _ = create_dataloader(
        data_dir=Path(config['data_dir']),
        split=config['split'],
        batch_size=config['batch_size'],
        num_workers=config['num_workers'],
        sequence_length=max_seq_length,
        mode='offline',
        shuffle=False
    )

    # Generate sequences
    print("\nGenerating sequences...")
    generated_sequences, ground_truth_sequences = generate_offline(
        model=model,
        dataloader=dataloader,
        tokenizer_info=tokenizer_info,
        device=device,
        temperature=config['temperature'],
        top_k=config['top_k']
    )

    # Calculate metrics
    print("\nCalculating metrics...")
    harmony_metrics = calculate_harmony_metrics(generated_sequences, tokenizer_info)
    emd_metrics = calculate_emd_metrics(generated_sequences, ground_truth_sequences, tokenizer_info)

    offline_metrics = {**harmony_metrics, **emd_metrics}

    print("\n=== Offline Model Results ===")
    for metric, value in offline_metrics.items():
        print(f"{metric}: {value:.4f}")
        wandb.log({f"offline/{metric}": value})
        
    # Log the run ID we evaluated
    wandb.log({"offline_model_run_id": config['offline_run_id']})
else:
    print("\n=== Skipping Offline Model Evaluation ===")
    print("No offline model checkpoint provided")
generated_sequences, ground_truth_sequences = generate_offline(
    model=model,
    dataloader=dataloader,
    tokenizer_info=tokenizer_info,
    device=device
)

# Calculate metrics
print("\nCalculating metrics...")
harmony_metrics = calculate_harmony_metrics(generated_sequences, tokenizer_info)
emd_metrics = calculate_emd_metrics(generated_sequences, ground_truth_sequences, tokenizer_info)

offline_metrics = {**harmony_metrics, **emd_metrics}

print("\n=== Offline Model Results ===")
for metric, value in offline_metrics.items():
    print(f"{metric}: {value:.4f}")
    wandb.log({f"offline/{metric}": value})

In [ ]:
# Compare Results
print("\n=== Model Comparison ===")
print(f"{'Metric':<30} {'Online':<10} {'Offline':<10} {'Difference':<10}")
print("-" * 60)

for metric in online_metrics.keys():
    online_value = online_metrics[metric]
    offline_value = offline_metrics[metric]
    diff = online_value - offline_value
    print(f"{metric:<30} {online_value:>10.4f} {offline_value:>10.4f} {diff:>10.4f}")
    wandb.log({
        f"comparison/{metric}_diff": diff,
        f"comparison/{metric}_ratio": online_value / offline_value if offline_value != 0 else 0
    })

# Finish wandb run
wandb.finish()